In [1]:
!git clone https://github.com/yujiafeng8888/scCNA.git
%cd scCNA
!pip install .

fatal: destination path 'scCNA' already exists and is not an empty directory.
/content/scCNA
Processing /content/scCNA
  Preparing metadata (setup.py) ... done
  Created wheel for scCNA: filename=scCNA-0.1-py3-none-any.whl size=5948 sha256=b9607474bc4552e4ebecb0c258c4d70c029b815c18c4ed9e063f4b40df3af278
  Stored in directory: /tmp/pip-ephem-wheel-cache-960nsau_/wheels/25/cc/c5/ae548f3a909d160258148b54a2721305eb2d68474f5650137d
Successfully built scCNA
  Attempting uninstall: scCNA
    Found existing installation: scCNA 0.1
    Uninstalling scCNA-0.1:
      Successfully uninstalled scCNA-0.1


In [2]:
!pip install scanpy

In [3]:
import scanpy as sc
import scCNA as cna
import numpy as np
import random
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, average_precision_score

In [4]:
from google.colab import drive

drive.mount('/content/drive', force_remount=True)
adata = sc.read_h5ad("/content/drive/MyDrive/PBMC_simulated_cnas_041025.h5ad")

Mounted at /content/drive


In [9]:
def simulate_cnas_windowed(
    adata,
    window_size=100,
    cna_types=("gain", "loss", "homo_del"),
    frequencies=(0.2, 0.5, 0.9),
    chroms=("1", "2", "3"),
    cell_key="cell_type"
):
    """
    Simulate CNAs in expression matrix by modifying expression of genomic windows.
    Writes `simulated_cnvs` to adata.obs (overwrites if exists).

    Parameters:
    - adata: AnnData object
    - window_size: number of genes per simulated CNA window
    - cna_types: list of CNA types to simulate
    - frequencies: list of cell-level frequencies for each simulated CNA
    - chroms: chromosomes to simulate CNAs on
    """
    if "simulated_cnvs" not in adata.obs.columns:
        adata.obs["simulated_cnvs"] = ""
    else:
        adata.obs["simulated_cnvs"] = ""

    if not isinstance(adata.X, np.ndarray):
        X = adata.X.toarray()
    else:
        X = adata.X

    var = adata.var
    simulated_log = []

    for chrom in chroms:
        genes_chr = var[var["chromosome"] == chrom]
        gene_indices = genes_chr.index.to_numpy()

        if len(gene_indices) < window_size:
            continue

        start_idx = random.randint(0, len(gene_indices) - window_size)
        window_genes = gene_indices[start_idx:start_idx + window_size]
        window_starts = var.loc[window_genes, "start"]
        region_start = window_starts.min()
        region_end = window_starts.max()

        # Choose CNA type and frequency
        cna_type = random.choice(cna_types)
        freq = random.choice(frequencies)
        n_cells = int(len(adata) * freq)
        selected_cells = np.random.choice(adata.obs_names, n_cells, replace=False)

        # Apply effect
        for cell in selected_cells:
            idx = adata.obs_names.get_loc(cell)
            if cna_type == "gain":
                X[idx, adata.var_names.isin(window_genes)] *= 1.5
            elif cna_type == "loss":
                X[idx, adata.var_names.isin(window_genes)] *= 0.5
            elif cna_type == "homo_del":
                X[idx, adata.var_names.isin(window_genes)] = 0

            region_str = f"{chrom}:{region_start}-{region_end} (CN {0 if cna_type == 'homo_del' else ('+' if cna_type == 'gain' else '-')})"
            if adata.obs.at[cell, "simulated_cnvs"] == "":
                adata.obs.at[cell, "simulated_cnvs"] = region_str
            else:
                adata.obs.at[cell, "simulated_cnvs"] += f";{region_str}"

        simulated_log.append((chrom, region_start, region_end, cna_type, freq))

    adata.X = X
    return adata, simulated_log


In [5]:
print(dir(cna))

['CNAfinder', '__builtins__', '__cached__', '__doc__', '__file__', '__loader__', '__name__', '__package__', '__path__', '__spec__', '__version__', 'annotate_gene_position', 'find_cnas', 'utils']


In [6]:
adata_sim = cna.simulate_cnas_windowed(
    adata.copy(),
    window_size=100,
    cna_types=["gain", "loss", "homo_del"],
    frequencies=[0.2, 0.5, 0.9],
    chroms=["1", "2", "3"]
)

AttributeError: module 'scCNA' has no attribute 'simulate_cnas_windowed'

In [10]:
adata_sim, log = simulate_cnas_windowed(
    adata.copy(),
    window_size=100,
    cna_types=["gain", "loss", "homo_del"],
    frequencies=[0.2, 0.5, 0.9],
    chroms=["1", "2", "3"]
)

In [11]:
display(adata_sim.obs.head())

,n_genes_by_counts,total_counts,total_counts_ribo,pct_counts_ribo,total_counts_mt,pct_counts_mt,n_genes,n_counts,cell_type,simulated_cnvs
AAACCCAAGCGCCCAT-1,1005,1760.0,392.0,17.785845,52.0,2.359347,1005,1760.0,CD4 T cell,1:214603185.0-229440259.0 (CN +);2:196151263.0...
AAACCCAAGGTTCCGC-1,4101,14240.0,4526.0,22.528622,1324.0,6.590343,4101,14240.0,Dendritic,1:214603185.0-229440259.0 (CN +);2:196151263.0...
AAACCCACAGAGTTGG-1,1742,4208.0,1043.0,17.726036,633.0,10.757988,1742,4208.0,CD14 monocyte,2:196151263.0-210030572.0 (CN -);3:99817837.0-...
AAACCCACAGGTATGG-1,2122,4354.0,742.0,13.417721,434.0,7.848101,2122,4354.0,NK cell,1:214603185.0-229440259.0 (CN +);2:196151263.0...
AAACCCACATAGTCAC-1,1521,2819.0,1734.0,33.960049,553.0,10.830396,1521,2819.0,B cell,1:214603185.0-229440259.0 (CN +);2:196151263.0...


In [12]:
y_true = (adata_sim.obs['simulated_cnvs'].notna()) & (adata_sim.obs['simulated_cnvs'] != '')

In [13]:
print(y_true.value_counts())

simulated_cnvs
True     10047
False      262
Name: count, dtype: int64


In [7]:
ad_def = cna.find_cnas(
    adata.copy(),
    reference_key='cell_type',
    reference_cat=[
        'CD4 T cell','CD14 monocyte','B cell','CD8 T cell',
        'NK cell','FCGR3A monocyte','Dendritic','Megakaryocyte'
    ],
    threshold=5,
    min_cells=20,
    window_size=100
)

/usr/local/lib/python3.11/dist-packages/scanpy/preprocessing/_normalization.py:216: UserWarning: Received a view of an AnnData. Making a copy.
  view_to_actual(adata)


KeyboardInterrupt: 

In [ ]:
y_true = (adata_sim.obs['simulated_cnvs'].notna()) & (adata_sim.obs['simulated_cnvs'] != '')
#y_pred = (ad_def.obs['detect_CNA'].notna()) & (ad_def.obs['detect_CNA'] != '')
y_pred = ad_def.obs['detect_CNA'].apply(lambda x: x != 'none' and isinstance(x, str))

print("=== Cell-level Binary Classification Metrics ===")
print("Accuracy :", accuracy_score(y_true, y_pred))
print("Precision:", precision_score(y_true, y_pred))
print("Recall   :", recall_score(y_true, y_pred))
print("F1 Score :", f1_score(y_true, y_pred))
print("PR-AUC   :", average_precision_score(y_true, y_pred))

NameError: name 'ad_sim' is not defined